In [1]:
import os
import numpy as np
import pandas as pd
import time
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn import metrics
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
np.random.seed(1)
import random as rn
rn.seed(1)
import tensorflow as tf
tf.random.set_seed(1)

# Load data
train = pd.read_csv(os.path.join(os.getcwd(),'Input', 'SonyAIBOSurface2_Train.csv'))
test = pd.read_csv(os.path.join(os.getcwd(),'Input', 'SonyAIBOSurface2_Test.csv'))

# Preprocess the data
x_train, y_train = train.iloc[:, :-1].values, train.iloc[:, -1].values
x_test, y_test = test.iloc[:, :-1].values, test.iloc[:, -1].values

# Feature scaling
sc = StandardScaler()
x_train = sc.fit_transform(x_train).reshape(x_train.shape[0], 65, 1)
x_test = sc.transform(x_test).reshape(x_test.shape[0], 65, 1)

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

# Set up directories and model parameters
base_directory = os.path.join(os.getcwd(), 'Output', 'LSTM')
os.makedirs(base_directory, exist_ok=True)

# Define model parameters and train
m, n, epochs, patience = 6, 6, 300, 3  # example values
subdirectory = os.path.join(base_directory, f"{m}.{n}.{epochs}.{patience}")
os.makedirs(subdirectory, exist_ok=True)

# Build the model
model = Sequential([
    LSTM(m, return_sequences=True, input_shape=(65, 1)),
    LSTM(n),
    Dense(2, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Callbacks
es = EarlyStopping(monitor='val_loss', mode='min', patience=patience)
cp = ModelCheckpoint(os.path.join(subdirectory, 'bestweights.keras'), monitor='val_loss', mode='min', save_best_only=True)

# Train the model
model.fit(x_train, y_train, validation_split=0.2, epochs=epochs, batch_size=12, callbacks=[es, cp])

# Evaluate the model
scores = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {scores[1] * 100:.2f}%")

# Predict and calculate testing time
start_time = time.time()
y_pred = model.predict(x_test)
end_time = time.time()
testing_time = end_time - start_time
print(f"Testing Time: {testing_time} seconds")

# Save testing time
with open(os.path.join(subdirectory, 'testing_time.txt'), "w") as f:
    f.write(f"Testing Time: {testing_time} seconds")

# Save class probabilities
np.savetxt(os.path.join(subdirectory, 'class_probabilities.csv'), y_pred, delimiter=',')

# Classification report and confusion matrix
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Save classification report
report_str = classification_report(y_test_labels, y_pred_labels)
with open(os.path.join(subdirectory, 'classification_report.txt'), "w") as f:
    f.write(report_str)

# Confusion matrix plot
disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test_labels, y_pred_labels)
disp.figure_.suptitle("Confusion Matrix")
disp.figure_.savefig(os.path.join(subdirectory, 'Confusion_Matrix.png'))
plt.close()


Epoch 1/300


C:\Users\tenzi\anaconda3\envs\aeon-env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.5294 - loss: 0.6913 - val_accuracy: 0.6178 - val_loss: 0.6647
Epoch 2/300
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6277 - loss: 0.6556 - val_accuracy: 0.6178 - val_loss: 0.6308
Epoch 3/300
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6455 - loss: 0.6055 - val_accuracy: 0.6963 - val_loss: 0.5747
Epoch 4/300
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7785 - loss: 0.4771 - val_accuracy: 0.7435 - val_loss: 0.5234
Epoch 5/300
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7957 - loss: 0.4366 - val_accuracy: 0.7592 - val_loss: 0.5115
Epoch 6/300
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8039 - loss: 0.4088 - val_accuracy: 0.7749 - val_loss: 0.4854
Epoch 7/300
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8267 - loss: 0.3850 - val_accuracy: 0.7696 - val_loss: 0.4666
Epoch 8/300
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8344 - loss: 0.3654 - val_accuracy: 0.7958 - val_l

In [4]:
x_train.shape

(953, 65)